In [1]:
def preprocessor(df1):
    import pandas as pd
    import numpy as np
    from sklearn import preprocessing
    #df1=df1[df1['Status']=='Overdue']
    #print(df1['Status'])
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    df2=df1[['Net.Due.Dt','Clearing','Status']]
    df2.dropna(inplace=True)
    df2['Range of Delay']=pd.to_datetime(df2['Clearing'],dayfirst=True)-pd.to_datetime(df2['Net.Due.Dt'],dayfirst=True)
    df2['Range of Delay']=df2['Range of Delay'].dt.days
    df2['Range of Delay']=pd.to_numeric(df2['Range of Delay'])
    df2 = df2.reset_index()
    df2.drop(columns = 'index',inplace=True)
    
    df2['Status'] = np.where(df2['Range of Delay'] > 15, 2, (np.where(df2['Range of Delay'] > 0, 1,0)))#(np.where(df2['Range of Delay'] > 0, 1,0)))))#(np.where(df2['Range of Delay'] > 0, 1,0)))))#(np.where(df2['Range of Delay'] > 10, 2,(np.where(df2['Range of Delay'] > 0,1,0)))))))# (np.where(df2['Range of Delay'] > 30, 5, (np.where(df2['Range of Delay'] > 15, 4, (np.where(df2['Range of Delay'] > 7, 3, (np.where(df2['Range of Delay'] > 3, 2, (np.where(df2['Range of Delay'] > 0, 1 ,0)))))))))))))))
    #print(df2)
    print('Y s in the initial set')
    print(np.unique(df2['Status'],return_counts=True))
    print('----------------------------------------------------------------------------------------')
    
    df1['Status']=df2['Status']
    df1.dropna(inplace=True)
    y=df1['Status']
    
    
    busa = preprocessing.LabelEncoder()
    ccar= preprocessing.LabelEncoder()
    month=preprocessing.LabelEncoder()
    zone=preprocessing.LabelEncoder()
    bran=preprocessing.LabelEncoder()
    payt=preprocessing.LabelEncoder()
    
    df1.drop(columns='Status',inplace=True)
    #df1['Status']=df1['Status'].astype('category').cat.codes
    df1['BusA']=busa.fit_transform(df1['BusA'])
    df1['CCAr']=ccar.fit_transform(df1['CCAr'])#.astype('category').cat.codes
    #df1['Account']=df1['Account'].astype('category').cat.codes
    df1['Month']=month.fit_transform(df1['Month'])#.astype('category').cat.codes
    #df1.drop(columns='Reference',inplace=True)
    df1.drop(columns='Customer.Name',inplace=True)
    df1.drop(columns=['DocumentNo','Year','Clrng.doc.'],inplace=True)
    df1['Zone']=zone.fit_transform(df1['Zone'])#.astype('category').cat.codes
    df1['Bran']=bran.fit_transform(df1['Bran'])#.astype('category').cat.codes
    df1['PayT'] =payt.fit_transform(df1['PayT'])#.astype('category').cat.codes
    df1.drop(columns='Doc.Chq.dt',inplace=True)
    df1.drop(columns='Ty',inplace=True)
    df1.drop(columns='Sale.Type',inplace=True)
    df1['Pstng.Date']=pd.to_datetime(df1['Pstng.Date'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    df1['Net.Due.Dt']=pd.to_datetime(df1['Net.Due.Dt'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    df1['Clearing']=pd.to_datetime(df1['Clearing']).dt.strftime("%Y%m%d").astype(str)
    df1.drop(columns='Clearing',inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    df1.drop(columns='Arr..Clearing...Net.Due.Date.',inplace=True)
    df1.drop(columns='G.L',inplace=True)
    
    #print('Creating Red Alert Zone')
    #table = pd.crosstab(columns=y, index=df1['Account'])
    #table.columns= ["No delay"," 0 - 10", ">10"]
    #print(table)
    #print(table.sort_values(['>10'],ascending=False).head())
    #arr=table.sort_values(['>10'],ascending=False)
    #arr.drop(columns=["No delay"," 0 - 10"],inplace=True)
    #df_1=arr[arr['>10']>40]
    #df_2=[]
    
    
    #df1['Account']=df1['Account']/100000
    #df1['Reference']=df1['Reference']/1000000000
    


    #print(df_1.index)
    
    
    ###################Oversampling##############################
    from imblearn.over_sampling import ADASYN 
    sm = ADASYN()
    df3, y = sm.fit_resample(df1, y)
    df1=pd.DataFrame(df3,columns=df1.columns)
    ############################################################
    
    #y1=df1.columns
    #x = df1.values #returns a numpy array
    #min_max_scaler = preprocessing.MinMaxScaler()
    #x_scaled = min_max_scaler.fit_transform(x)
    #df1 = pd.DataFrame(x_scaled,columns=df1.columns)
    
    print('Y s in the set')
    print(np.unique(y,return_counts=True))
    print('----------------------------------------------------------------------------------------')
    
    return df1,y,busa,ccar,month,zone,bran,payt

In [2]:
def trainer(df1,y):
    import pandas as pd
    import numpy as np
    from sklearn import metrics
    from sklearn.ensemble import RandomForestClassifier
    from sklearn import tree
    from sklearn import neighbors
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.ensemble import VotingClassifier
    from sklearn.ensemble import BaggingClassifier
    from sklearn.model_selection import train_test_split
    
    #y=np.array(df1['Status'])
    
    

    X=df1
    #df1.head(5)
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3, random_state=0)
    # Random forest Model: 
    model= RandomForestClassifier(n_estimators = 300,random_state = 30,oob_score=True,n_jobs=-1,verbose=1,max_features=None,class_weight={0:10,1:20,2:1000})
    # Decion Tree Model: 
    decision_tree = tree.DecisionTreeClassifier(random_state=0, max_depth=100)
    # AdaBoost Classifier Model:
    clf = AdaBoostClassifier(base_estimator=model,n_estimators=100, random_state=0)
    # KNN :
    mo = neighbors.KNeighborsClassifier(n_neighbors =2)
    # Bagging Classifier Model:  
    bagging = BaggingClassifier(mo,max_samples=0.5, max_features=0.5)
    #Voting Classifier
    eclf = model#VotingClassifier(estimators=[ ('knn_bagging', bagging), ('adaboost', clf), ('RF',model)],voting='soft')
    eclf1 = eclf.fit(xtrain, ytrain)
    print('TRAINING DONE ......................')
    eclf1=eclf.predict(xtrain)
    eclf2=eclf.predict(xtest)
    print('TRAINING RESULTS-----------------------------------------')
    print("Accuracy: for Training :",metrics.accuracy_score(ytrain, eclf1)*100)
    print("Accuracy: for Testing :",metrics.accuracy_score(ytest, eclf2)*100)
    print()
    print('Confusion Matrix for Random Forest')
    print(metrics.confusion_matrix(eclf2, ytest))
    print()
    print('--------------------------------------------------------------------------------------------------------------------')
    return eclf

In [3]:
def predict(df1,clf,busa,ccar,month,zone,bran,payt):
    import numpy as np
    import pandas as pd
    import copy
    from sklearn import preprocessing
    df=copy.deepcopy(df1)
    print(df.shape)
    df['Prediction']=np.zeros(len(df1))
    print(df1.shape)
    df1.dropna(inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    #df1['Status']=df1['Status'].astype('category').cat.codes)
    #df1['Status']=df1['Status'].astype('category').cat.codes
    busa = preprocessing.LabelEncoder()
    ccar= preprocessing.LabelEncoder()
    month=preprocessing.LabelEncoder()
    zone=preprocessing.LabelEncoder()
    bran=preprocessing.LabelEncoder()
    payt=preprocessing.LabelEncoder()
    

    #df1['Status']=df1['Status'].astype('category').cat.codes
    df1['BusA']=busa.fit_transform(df1['BusA'])
    df1['CCAr']=ccar.fit_transform(df1['CCAr'])#.astype('category').cat.codes
    #df1['Account']=df1['Account'].astype('category').cat.codes
    df1['Month']=month.fit_transform(df1['Month'])#.astype('category').cat.codes
    #df1.drop(columns='Reference',inplace=True)
    df1.drop(columns='Customer.Name',inplace=True)
    df1.drop(columns=['DocumentNo','Year'],inplace=True)
    df1['Zone']=zone.fit_transform(df1['Zone'])#.astype('category').cat.codes
    df1['Bran']=bran.fit_transform(df1['Bran'])#.astype('category').cat.codes
    df1['PayT'] =payt.fit_transform(df1['PayT'])#.astype('category').cat.codes
    df1.drop(columns='Doc.Chq.dt',inplace=True)
    df1.drop(columns='Ty',inplace=True)
    df1.drop(columns='Sale.Type',inplace=True)
    df1['Pstng.Date']=pd.to_datetime(df1['Pstng.Date'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    df1['Net.Due.Dt']=pd.to_datetime(df1['Net.Due.Dt'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    #df1['Clearing']=pd.to_datetime(df1['Clearing']).dt.strftime("%Y%m%d").astype(str)
    #df1.drop(columns=['Net.Due.Dt','Clearing'],inplace=True)
    #df1.drop(columns='Net.Due.Dt',inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    df1.drop(columns='Arr..Clearing...Net.Due.Date.',inplace=True)
    df1.drop(columns='G.L',inplace=True)
    
    #y1=df1.columns
    #x = df1.values #returns a numpy array
    #min_max_scaler = preprocessing.MinMaxScaler()
    #x_scaled = min_max_scaler.fit_transform(x)
    #df1 = pd.DataFrame(x_scaled,columns=df1.columns)
    
    #df1['Account']=df1['Account']/100000
    #df1['Reference']=df1['Reference']/1000000000
    
    
    print('Starting Prediction ....')
    prediction=clf.predict(df1)
    print('Generating Probabilities.....')
    predictprob=clf.predict_proba(df1)
    print('Finished prediction...')
    print()
    #df['Probability']=np.zeros(len(df))
    print(df1.head())
    

    arr=['No Delay','Low Delay','High Delay']
    arr1=['Low delay','Low Medium Delay','High Delay']#,'High Delay']#,'30 to 180','>180']#,'8-15 days','16-30 days','31-60 days','61-90 days','90-180 days','>180 days']
    for i in range(len(arr)):
        df[arr1[i]]=np.empty(len(df))
    print('Computing Prediction Table............')
    #for i in range(len(prediction)):
        #flag=0
        #if ( i % 500 == 0):
         #   print('----------------------------------------------------------------------------------------------------------- i =',i)
        #for k in range(len(redzone)):
           # if df['Account'][i] == redzone[k] :
             #   df['Prediction'][i]='>=10'
              #  flag=1
              #  print('----------------------------------------------------------------RED ALERT------------------------------------------ ')
                
        #if(flag == 0):
    
        #print(arr[prediction[i]])
        #arr2=""
        #print('Actual:',df['Arr..Clearing...Net.Due.Date.'][i])
        #print('--------------------------------------------------------')
    df['Prediction']=prediction
    df['Prediction']=np.where(df['Prediction'] == 0, 'No Delay',(np.where(df['Prediction'] == 1, 'Low Delay',(np.where(df['Prediction'] == 2, 'High Delay','')))))
    for i in range(len(arr1)):
        df[arr1[i]]=np.around(predictprob[:,i],decimals=2)
        
    print(df.head(5))
            
    print()
    print(df.head())
    print('Writing to csv file')
    print('-----------------------------------------------------------------------------------------')
    df.to_csv('Predictedout5.csv',index=False)

In [4]:
# import time
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import copy
trainpath='MyData.csv'

print('Reading data...')
df1=pd.read_csv(trainpath)
print('Finished Reading...')
print()
print(df1.head(5))
print()

df3=copy.deepcopy(df1)
print('Preprocessing started .......')
df2,y,busa,ccar,month,zone,bran,payt=preprocessor(df1)
print(df2.head(5))
print('Preprocessing Done...........')
print('--------------------------------------------------------------------------------------------------------')
#print('RED ZONE VISUALISATION....')

#visualisation_red(df3)

Reading data...
Finished Reading...

   BusA CCAr     Reference  Account          Customer.Name   Zone  Bran  Ty  \
0  1000  BPR  4.132000e+09   929488      KAUSALYA AGENCIES  South  SBAD  RV   
1  9600  BPR  1.115502e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   
2  9600  BPR  1.115502e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   
3  1000  BPR  2.412166e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   
4  2500  BPR  2.410320e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   

    Status  Local.Crcy.Amt  ...  Doc.Chq.dt Month Pstng.Date  Net.Due.Dt  \
0   Not OD            0.10  ...  31.12.2018   Dec   31-12-18    05-01-19   
1  Overdue            0.04  ...  31.10.2018   Oct   31-10-18    10-11-18   
2   Not OD            0.12  ...  05.11.2018   Nov   05-11-18    15-11-18   
3  Overdue            0.17  ...  30.12.2018   Dec   30-12-18    09-01-19   
4  Overdue            0.16  ...  03.11.2018   Nov   03-11-18    13-11-18   

       G.L  Year Clrng.doc.  Cl

In [5]:
import time
t1=time.time()
print('Training Starting............')
clf=trainer(df2,y)
print('Training Done...................')
print('Time Taken',time.time()-t1)
print('-----------------------------------------------------------------------------------')

Training Starting............


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 18.9min finished


TRAINING DONE ......................


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    2.7s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   11.8s
[Parallel(n_jobs=2)]: Done 300 out of 300 | elapsed:   18.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    2.2s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    9.3s
[Parallel(n_jobs=2)]: Done 300 out of 300 | elapsed:   14.7s finished


TRAINING RESULTS-----------------------------------------
Accuracy: for Training : 99.65157163994675
Accuracy: for Testing : 97.36057317510797

Confusion Matrix for Random Forest
[[25898   698    35]
 [ 1187 25775    40]
 [   38   135 27007]]

--------------------------------------------------------------------------------------------------------------------
Training Done...................
Time Taken 1187.8959438800812
-----------------------------------------------------------------------------------


In [6]:
print('Feature Importances')
for feature in zip(df2.columns, clf.feature_importances_):
    print(feature)

Feature Importances
('BusA', 0.04039194374611351)
('CCAr', 0.0)
('Reference', 0.14683947545504744)
('Account', 0.13148644030481524)
('Zone', 0.07628144365015575)
('Bran', 0.11761115048602398)
('Local.Crcy.Amt', 0.06711158629359033)
('PayT', 0.04828126390305931)
('Month', 0.10575127149788208)
('Pstng.Date', 0.034889041429202565)
('Net.Due.Dt', 0.12680460634917434)
('Pending_Amy', 0.054022944173949636)
('pending_cnt', 0.05052883271098587)


In [7]:
testpath='Test.csv'
test=pd.read_csv(testpath)
predict(test,clf,busa,ccar,month,zone,bran,payt)
print() 
print('______________________________________  END  ____________________________________')
print()
print()

(5273, 21)
(5273, 21)
Starting Prediction ....


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 300 out of 300 | elapsed:    0.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


Generating Probabilities.....


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 300 out of 300 | elapsed:    0.6s finished


Finished prediction...

   BusA  CCAr     Reference  Account  Zone  Bran  Local.Crcy.Amt  PayT  Month  \
0     1     0  2.412118e+09   926346     1     8            0.14     8      0   
1     1     0  2.412118e+09   926346     1     8            0.15     8      0   
2     1     0  2.412118e+09   926346     1     8            0.15     8      0   
3     1     0  2.412118e+09   926346     1     8            0.16     8      0   
4     1     0  2.412118e+09   926346     1     8            0.16     8      0   

   Pstng.Date  Net.Due.Dt  Pending_Amy  pending_cnt  
0    20180901    20180911         1.49           11  
1    20180901    20180911         1.49           11  
2    20180901    20180911         1.49           11  
3    20180901    20180911         1.49           11  
4    20180901    20180911         1.49           11  
Computing Prediction Table............
   BusA CCAr     Reference  Account         Customer.Name   Zone  Bran  Ty  \
0  2000  BPR  2.412118e+09   926346  GRG STEELS 